In [21]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.discrete.discrete_model import Logit
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.graphics.gofplots import ProbPlot


In [22]:
class CreditRiskModelingStatsmodels:
    def __init__(self):
        self.pd_model = None
        self.lgd_model = None
        self.ead_model = None
        self.pd_features = None
        self.lgd_features = None
        self.ead_features = None
        
    def load_data(self, filepath):
        """
        Load credit data from CSV file
        """
        try:
            data = pd.read_csv(filepath)
            print(f"Data loaded successfully with shape: {data.shape}")
            return data
        except Exception as e:
            print(f"Error loading data: {e}")
            return None

    def preprocess_data(self, data, target_column='default_flag'):
        """
        Preprocess data for modeling
        """
        # Make a copy to avoid modifying the original
        processed_data = data.copy()
        
        # Handle missing values
        numeric_cols = processed_data.select_dtypes(include=['float64', 'int64']).columns
        processed_data[numeric_cols] = processed_data[numeric_cols].fillna(processed_data[numeric_cols].median())
        
        categorical_cols = processed_data.select_dtypes(include=['object']).columns
        for col in categorical_cols:
            processed_data[col] = processed_data[col].fillna(processed_data[col].mode()[0])
        
        # Create dummy variables for categorical columns
        processed_data = pd.get_dummies(processed_data, columns=categorical_cols, drop_first=True)
        
        # Split data into train and test
        np.random.seed(42)
        mask = np.random.rand(len(processed_data)) < 0.8
        train_data = processed_data[mask]
        test_data = processed_data[~mask]
        
        print(f"Training data shape: {train_data.shape}")
        print(f"Test data shape: {test_data.shape}")
        
        return train_data, test_data
    
    def check_multicollinearity(self, X):
        """
        Check for multicollinearity using Variance Inflation Factor (VIF)
        """
        X_with_const = add_constant(X)
        vif_data = pd.DataFrame()
        vif_data["Feature"] = X_with_const.columns
        vif_data["VIF"] = [variance_inflation_factor(X_with_const.values, i) for i in range(X_with_const.shape[1])]
        
        print("Variance Inflation Factors:")
        print(vif_data.sort_values('VIF', ascending=False))
        
        # Flag problematic features
        high_vif_features = vif_data[vif_data["VIF"] > 10]["Feature"].tolist()
        if "const" in high_vif_features:
            high_vif_features.remove("const")
            
        if high_vif_features:
            print(f"Warning: High multicollinearity detected in features: {high_vif_features}")
        
        return vif_data
    
    def train_pd_model(self, train_data, features, target='default_flag'):
        """
        Train Probability of Default model using Logistic Regression (Logit)
        """
        # Store feature names for later prediction
        self.pd_features = features
        
        # Prepare data
        X = train_data[features]
        X = add_constant(X)
        y = train_data[target]
        
        # Check multicollinearity 
        self.check_multicollinearity(train_data[features])
        
        # Fit model
        logit_model = Logit(y, X)
        try:
            self.pd_model = logit_model.fit(disp=0)  # disp=0 suppresses convergence messages
            print("\nPD Model Summary:")
            print(self.pd_model.summary())
            
            # Analyze p-values
            params = self.pd_model.params
            pvalues = self.pd_model.pvalues
            print("\nSignificant features (p < 0.05):")
            for feature, pvalue in zip(self.pd_model.model.exog_names, pvalues):
                if pvalue < 0.05:
                    print(f"{feature}: coefficient = {params[feature]:.4f}, p-value = {pvalue:.4f}")
            
            return self.pd_model
        except Exception as e:
            print(f"Error fitting PD model: {e}")
            return None
        
    def train_lgd_model(self, train_data, features, target='lgd', defaulted_only=True):
        """
        Train Loss Given Default model using OLS regression
        Default is to train only on defaulted loans
        """
        # Store feature names for later prediction
        self.lgd_features = features
        
        # Filter for defaulted loans if requested
        if defaulted_only:
            train_subset = train_data[train_data['default_flag'] == 1].copy()
            if len(train_subset) < 10:
                print("Warning: Not enough defaulted loans for LGD modeling")
                return None
        else:
            train_subset = train_data.copy()
            
        # Prepare data
        X = train_subset[features]
        X = add_constant(X)
        y = train_subset[target]
        
        # Check multicollinearity
        self.check_multicollinearity(train_subset[features])
        
        # Fit model using OLS
        try:
            self.lgd_model = sm.OLS(y, X).fit()
            print("\nLGD Model Summary:")
            print(self.lgd_model.summary())
            
            # Diagnostic plots
            self.lgd_diagnostic_plots(self.lgd_model)
            
            # Calculate robust errors if there's heteroscedasticity
            lgd_model_robust = sm.OLS(y, X).fit(cov_type='HC3')
            print("\nLGD Model with Heteroscedasticity-Robust Standard Errors:")
            print(lgd_model_robust.summary())
            
            return self.lgd_model
        except Exception as e:
            print(f"Error fitting LGD model: {e}")
            return None
    
    def train_ead_model(self, train_data, features, target_factor='ead_factor'):
        """
        Train Exposure at Default model using OLS regression
        EAD is modeled as a factor of outstanding balance
        """
        # Store feature names for later prediction
        self.ead_features = features
        
        # Prepare data
        X = train_data[features]
        X = add_constant(X)
        y = train_data[target_factor]
        
        # Check multicollinearity
        self.check_multicollinearity(train_data[features])
        
        # Fit model
        try:
            self.ead_model = sm.OLS(y, X).fit()
            print("\nEAD Model Summary:")
            print(self.ead_model.summary())
            
            # Diagnostic plots
            self.ead_diagnostic_plots(self.ead_model)
            
            return self.ead_model
        except Exception as e:
            print(f"Error fitting EAD model: {e}")
            return None
    
    def lgd_diagnostic_plots(self, model):
        """
        Create diagnostic plots for LGD OLS model
        """
        # Get model results
        y = model.model.endog
        y_pred = model.fittedvalues
        resid = model.resid
        
        # Create figure with 4 subplots
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        
        # Residuals vs Fitted
        axes[0, 0].scatter(y_pred, resid, edgecolors='k', facecolors='none')
        axes[0, 0].axhline(y=0, color='k', linestyle='--')
        axes[0, 0].set_xlabel('Fitted values')
        axes[0, 0].set_ylabel('Residuals')
        axes[0, 0].set_title('Residuals vs Fitted')
        
        # QQ plot
        qq = ProbPlot(resid)
        qq.qqplot(line='45', ax=axes[0, 1])
        axes[0, 1].set_title('Q-Q Plot')
        
        # Scale-Location
        axes[1, 0].scatter(y_pred, np.sqrt(np.abs(resid)), edgecolors='k', facecolors='none')
        axes[1, 0].set_xlabel('Fitted values')
        axes[1, 0].set_ylabel('√|Residuals|')
        axes[1, 0].set_title('Scale-Location Plot')
        
        # Actual vs Predicted
        axes[1, 1].scatter(y, y_pred, edgecolors='k', facecolors='none')
        min_val = min(min(y), min(y_pred))
        max_val = max(max(y), max(y_pred))
        axes[1, 1].plot([min_val, max_val], [min_val, max_val], 'k--')
        axes[1, 1].set_xlabel('Observed values')
        axes[1, 1].set_ylabel('Predicted values')
        axes[1, 1].set_title('Observed vs Predicted')
        
        plt.tight_layout()
        plt.show()
    
    def ead_diagnostic_plots(self, model):
        """ 
        Create diagnostic plots for EAD OLS model
        """
        # Same as LGD diagnostic plots
        self.lgd_diagnostic_plots(model)
    
    def evaluate_pd_model(self, test_data, target='default_flag'):
        """
        Evaluate PD model
        """
        if self.pd_model is None:
            print("PD model not trained yet")
            return None
        
        # Prepare data
        X_test = test_data[self.pd_features]
        X_test = add_constant(X_test)
        y_test = test_data[target]
        
        # Predict probabilities
        y_pred_proba = self.pd_model.predict(X_test)
        
        # Find optimal threshold using Youden's J statistic
        thresholds = np.arange(0.1, 0.9, 0.02)
        sensitivity = []
        specificity = []
        
        for threshold in thresholds:
            y_pred = (y_pred_proba >= threshold).astype(int)
            # True positive rate (TPR) = sensitivity
            tpr = sum((y_test == 1) & (y_pred == 1)) / sum(y_test == 1) if sum(y_test == 1) > 0 else 0
            # True negative rate (TNR) = specificity
            tnr = sum((y_test == 0) & (y_pred == 0)) / sum(y_test == 0) if sum(y_test == 0) > 0 else 0
            
            sensitivity.append(tpr)
            specificity.append(tnr)
        
        # Calculate Youden's J statistic (J = sensitivity + specificity - 1)
        j_scores = np.array(sensitivity) + np.array(specificity) - 1
        best_idx = np.argmax(j_scores)
        optimal_threshold = thresholds[best_idx]
        
        # Apply optimal threshold
        y_pred = (y_pred_proba >= optimal_threshold).astype(int)
        
        # Calculate performance metrics
        confusion = pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'])
        
        # Calculate metrics
        TP = confusion.loc[1, 1] if 1 in confusion.index and 1 in confusion.columns else 0
        TN = confusion.loc[0, 0] if 0 in confusion.index and 0 in confusion.columns else 0
        FP = confusion.loc[0, 1] if 0 in confusion.index and 1 in confusion.columns else 0
        FN = confusion.loc[1, 0] if 1 in confusion.index and 0 in confusion.columns else 0
        
        accuracy = (TP + TN) / (TP + TN + FP + FN)
        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        
        # Calculate AUC (Area Under ROC Curve)
        from sklearn.metrics import roc_curve, auc
        fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
        roc_auc = auc(fpr, tpr)
        
        # Print results
        print("\nPD Model Evaluation:")
        print(f"Optimal threshold: {optimal_threshold:.4f}")
        print(f"Confusion Matrix:\n{confusion}")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall (Sensitivity): {recall:.4f}")
        print(f"F1 Score: {f1:.4f}")
        print(f"AUC: {roc_auc:.4f}")
        
        # Plot ROC curve
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
        plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic (ROC) Curve')
        plt.legend(loc="lower right")
        plt.show()
        
        # Predicted vs Actual probability plot (calibration)
        plt.figure(figsize=(10, 6))
        
        # Bin the predictions
        bins = 10
        bin_edges = np.linspace(0, 1, bins + 1)
        bin_indices = np.digitize(y_pred_proba, bin_edges[1:-1])
        
        bin_pred_probs = []
        bin_actual_probs = []
        bin_counts = []
        
        for i in range(bins):
            mask = bin_indices == i
            if sum(mask) > 0:  # Only consider bins with data
                bin_pred_prob = np.mean(y_pred_proba[mask])
                bin_actual_prob = np.mean(y_test[mask])
                bin_count = sum(mask)
                
                bin_pred_probs.append(bin_pred_prob)
                bin_actual_probs.append(bin_actual_prob)
                bin_counts.append(bin_count)
        
        plt.scatter(bin_pred_probs, bin_actual_probs, s=[100 * c/sum(bin_counts) for c in bin_counts])
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlabel('Predicted Probability')
        plt.ylabel('Actual Probability')
        plt.title('Calibration Plot (Reliability Diagram)')
        plt.grid(True)
        plt.show()
        
        return {
            'threshold': optimal_threshold,
            'confusion_matrix': confusion,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'auc': roc_auc
        }
    
    def evaluate_lgd_model(self, test_data, target='lgd', defaulted_only=True):
        """
        Evaluate LGD model
        """
        if self.lgd_model is None:
            print("LGD model not trained yet")
            return None
            
        # Filter for defaulted loans if requested
        if defaulted_only:
            test_subset = test_data[test_data['default_flag'] == 1].copy()
            if len(test_subset) < 5:
                print("Warning: Not enough defaulted loans in test data for LGD evaluation")
                return None
        else:
            test_subset = test_data.copy()
        
        # Prepare data
        X_test = test_subset[self.lgd_features]
        X_test = add_constant(X_test)
        y_test = test_subset[target]
        
        # Predict
        y_pred = self.lgd_model.predict(X_test)
        
        # Clip predictions to valid range [0, 1]
        y_pred = np.clip(y_pred, 0, 1)
        
        # Calculate metrics
        mse = np.mean((y_test - y_pred) ** 2)
        rmse = np.sqrt(mse)
        mae = np.mean(np.abs(y_test - y_pred))
        r_squared = self.lgd_model.rsquared
        
        # Print results
        print("\nLGD Model Evaluation:")
        print(f"Mean Squared Error (MSE): {mse:.4f}")
        print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
        print(f"Mean Absolute Error (MAE): {mae:.4f}")
        print(f"R-squared: {r_squared:.4f}")
        
        # Plot actual vs predicted
        plt.figure(figsize=(10, 6))
        plt.scatter(y_test, y_pred, alpha=0.5, edgecolors='k')
        plt.plot([0, 1], [0, 1], 'r--')
        plt.xlim([0, 1])
        plt.ylim([0, 1])
        plt.xlabel('Actual LGD')
        plt.ylabel('Predicted LGD')
        plt.title('LGD Model: Actual vs Predicted')
        plt.grid(True)
        plt.show()
        
        return {
            'mse': mse,
            'rmse': rmse,
            'mae': mae,
            'r_squared': r_squared
        }
    
    def evaluate_ead_model(self, test_data, target_factor='ead_factor'):
        """
        Evaluate EAD model
        """
        if self.ead_model is None:
            print("EAD model not trained yet")
            return None
        
        # Prepare data
        X_test = test_data[self.ead_features]
        X_test = add_constant(X_test)
        y_test = test_data[target_factor]
        
        # Predict
        y_pred = self.ead_model.predict(X_test)
        
        # Clip predictions to reasonable range (e.g., [0.5, 1.5])
        y_pred = np.clip(y_pred, 0.5, 1.5)
        
        # Calculate metrics
        mse = np.mean((y_test - y_pred) ** 2)
        rmse = np.sqrt(mse)
        mae = np.mean(np.abs(y_test - y_pred))
        r_squared = self.ead_model.rsquared
        
        # Print results
        print("\nEAD Model Evaluation:")
        print(f"Mean Squared Error (MSE): {mse:.4f}")
        print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
        print(f"Mean Absolute Error (MAE): {mae:.4f}")
        print(f"R-squared: {r_squared:.4f}")
        
        # Plot actual vs predicted
        plt.figure(figsize=(10, 6))
        plt.scatter(y_test, y_pred, alpha=0.5, edgecolors='k')
        min_val = min(min(y_test), min(y_pred))
        max_val = max(max(y_test), max(y_pred))
        plt.plot([min_val, max_val], [min_val, max_val], 'r--')
        plt.xlabel('Actual EAD Factor')
        plt.ylabel('Predicted EAD Factor')
        plt.title('EAD Model: Actual vs Predicted')
        plt.grid(True)
        plt.show()
        
        return {
            'mse': mse,
            'rmse': rmse,
            'mae': mae,
            'r_squared': r_squared
        }
    
    def predict_pd(self, data):
        """
        Predict probability of default
        """
        if self.pd_model is None:
            print("PD model not trained yet")
            return None
        
        # Prepare features
        X = data[self.pd_features]
        X = add_constant(X)
        
        # Predict probabilities
        pd_scores = self.pd_model.predict(X)
        return pd_scores
    
    def predict_lgd(self, data):
        """
        Predict loss given default
        """
        if self.lgd_model is None:
            print("LGD model not trained yet")
            return None
        
        # Prepare features
        X = data[self.lgd_features]
        X = add_constant(X)
        
        # Predict
        lgd_scores = self.lgd_model.predict(X)
        # Ensure LGD is between 0 and 1
        lgd_scores = np.clip(lgd_scores, 0, 1)
        return lgd_scores
    
    def predict_ead(self, data, outstanding_amounts):
        """
        Predict exposure at default
        """
        if self.ead_model is None:
            print("EAD model not trained yet")
            return None
        
        # Prepare features
        X = data[self.ead_features]
        X = add_constant(X)
        
        # Predict EAD factor
        ead_factors = self.ead_model.predict(X)
        # Ensure EAD factor is reasonable
        ead_factors = np.clip(ead_factors, 0.5, 1.5)
        
        # Calculate absolute EAD amounts
        ead_amounts = ead_factors * outstanding_amounts
        return ead_amounts
    
    def calculate_expected_loss(self, pd_scores, lgd_scores, ead_amounts):
        """
        Calculate Expected Loss (EL) = PD * LGD * EAD
        """
        expected_loss = pd_scores * lgd_scores * ead_amounts
        return expected_loss
    
    def calculate_portfolio_metrics(self, expected_loss, confidence_levels=[0.95, 0.99]):
        """
        Calculate portfolio-level risk metrics
        """
        total_el = expected_loss.sum()
        
        metrics = {'Total Expected Loss': total_el}
        
        # Calculate VaR and CVaR at different confidence levels
        for cl in confidence_levels:
            var = np.percentile(expected_loss, cl * 100)
            metrics[f'VaR ({cl*100}%)'] = var
            
            # Calculate CVaR (Expected Shortfall)
            cvar = expected_loss[expected_loss >= var].mean()
            metrics[f'CVaR ({cl*100}%)'] = cvar
        
        # Calculate distribution metrics
        metrics['Mean EL'] = expected_loss.mean()
        metrics['Median EL'] = np.median(expected_loss)
        metrics['EL Standard Deviation'] = expected_loss.std()
        metrics['EL Skewness'] = stats.skew(expected_loss)
        metrics['EL Kurtosis'] = stats.kurtosis(expected_loss)
        
        # Print results
        print("\nPortfolio Risk Metrics:")
        for metric, value in metrics.items():
            print(f"{metric}: ${value:.2f}")
        
        return metrics
    
    def visualize_risk_distribution(self, expected_loss, confidence_level=0.95):
        """
        Visualize the distribution of expected loss
        """
        plt.figure(figsize=(12, 8))
        
        # Distribution plot
        sns.histplot(expected_loss, kde=True, stat="density", linewidth=0)
        
        # Add vertical lines for key metrics
        mean_el = expected_loss.mean()
        median_el = np.median(expected_loss)
        var_el = np.percentile(expected_loss, confidence_level * 100)
        cvar_el = expected_loss[expected_loss >= var_el].mean()
        
        plt.axvline(mean_el, color='r', linestyle='-', label=f'Mean: ${mean_el:.2f}')
        plt.axvline(median_el, color='g', linestyle='--', label=f'Median: ${median_el:.2f}')
        plt.axvline(var_el, color='b', linestyle='-.', 
                   label=f'{confidence_level*100}% VaR: ${var_el:.2f}')
        plt.axvline(cvar_el, color='m', linestyle=':', 
                   label=f'{confidence_level*100}% CVaR: ${cvar_el:.2f}')
        
        plt.xlabel('Expected Loss ($)')
        plt.ylabel('Density')
        plt.title('Distribution of Expected Loss')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()
        
        # QQ Plot for expected loss
        plt.figure(figsize=(8, 6))
        stats.probplot(expected_loss, dist="norm", plot=plt)
        plt.title('QQ Plot of Expected Loss')
        plt.grid(True, alpha=0.3)
        plt.show()


def generate_sample_data(n_samples=1000):
    """
    Generate sample credit risk data for demonstration
    """
    np.random.seed(42)
    
    # Borrower characteristics
    age = np.random.normal(40, 10, n_samples)
    income = np.random.lognormal(10, 0.5, n_samples) * 1000
    debt_to_income = np.random.beta(2, 5, n_samples)
    credit_score = np.random.normal(700, 100, n_samples)
    loan_amount = np.random.lognormal(10, 0.7, n_samples) * 1000
    loan_term = np.random.choice([12, 24, 36, 48, 60], n_samples)
    
    # Employment status
    employment_status = np.random.choice(['Employed', 'Self-employed', 'Unemployed', 'Retired'], 
                                          n_samples, p=[0.7, 0.2, 0.05, 0.05])
    
    # Create probability of default (influenced by features)
    pd_score = (0.3 * stats.norm.cdf(-credit_score, loc=600, scale=150) + 
                0.3 * debt_to_income + 
                0.2 * (np.log(loan_amount) - np.log(income)) + 
                0.2 * np.random.normal(0, 0.1, n_samples))
    pd_score = np.clip(pd_score, 0.001, 0.999)
    
    # Default flag based on probability
    default_flag = np.random.binomial(1, pd_score)
    
    # LGD only relevant for defaulted loans
    lgd = np.zeros(n_samples)
    defaulted = default_flag == 1
    lgd[defaulted] = 0.4 + 0.3 * np.random.beta(2, 2, sum(defaulted)) + 0.1 * debt_to_income[defaulted]
    lgd = np.clip(lgd, 0, 1)
    
    # EAD as a percentage of loan amount
    ead_factor = 0.8 + 0.2 * np.random.beta(5, 2, n_samples)
    outstanding_balance = loan_amount * np.random.uniform(0.3, 1.0, n_samples)  # Simulate partially paid loans
    ead_amount = ead_factor * outstanding_balance
    
    # Create DataFrame
    data = pd.DataFrame({
        'age': age,
        'income': income,
        'debt_to_income': debt_to_income,
        'credit_score': credit_score,
        'loan_amount': loan_amount,
        'loan_term': loan_term,
        'employment_status': employment_status,
        'outstanding_balance': outstanding_balance,
        'default_flag': default_flag,
        'lgd': lgd,
        'ead_factor': ead_factor,
        'ead_amount': ead_amount
    })
    
    return data


def main():
    """
    Main function to demonstrate credit risk modeling using statsmodels
    """
    # Either load your own data or generate sample data
    # data = load_data('your_credit_data.csv')
    data = generate_sample_data(n_samples=5000)
    
    # Initialize the credit risk model
    crm = CreditRiskModelingStatsmodels()
    
    # Preprocess data
    train_data, test_data = crm.preprocess_data(data, target_column='default_flag')
    
    # Define features for each model
    pd_features = ['age', 'income', 'debt_to_income', 'credit_score', 
                  'loan_amount', 'loan_term', 'employment_status_Employed', 
                  'employment_status_Retired', 'employment_status_Self-employed']
    
    lgd_features = ['debt_to_income', 'loan_amount', 'loan_term', 
                   'credit_score', 'employment_status_Employed']
    
    ead_features = ['loan_term', 'debt_to_income', 'employment_status_Employed',
                   'employment_status_Self-employed']
    
    # Train models
    crm.train_pd_model(train_data, pd_features, target='default_flag')
    crm.train_lgd_model(train_data, lgd_features, target='lgd', defaulted_only=True)
    crm.train_ead_model(train_data, ead_features, target_factor='ead_factor')
    
    # Evaluate models
    crm.evaluate_pd_model(test_data, target='default_flag')
    crm.evaluate_lgd_model(test_data, target='lgd', defaulted_only=True)
    crm.evaluate_ead_model(test_data, target_factor='ead_factor')
    
    # Make predictions on test data
    pd_scores = crm.predict_pd(test_data)
    lgd_scores = crm.predict_lgd(test_data)
    ead_amounts = crm.predict_ead(test_data, test_data['outstanding_balance'])
    
    # Calculate expected loss
    expected_loss = crm.calculate_expected_loss(pd_scores, lgd_scores, ead_amounts)
    
    # Analyze portfolio risk
    portfolio_metrics = crm.calculate_portfolio_metrics(expected_loss)
    crm.visualize_risk_distribution(expected_loss)

if __name__ == "__main__":
    main()

Training data shape: (4001, 14)
Test data shape: (999, 14)


KeyError: "['employment_status_Employed'] not in index"